<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/ML/%D0%A2%D0%B5%D0%BC%D0%B0_2_Boosting_Tree_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##**AdaBoost (Adaptive Boosting)**  

**Краткое описание:**  
Метод Adaptive Boosting (AdaBoost) представляет собой процесс последовательного добавления слабых базовых моделей к существующему ансамблю, с адаптивной корректировкой весов для каждой из базовых моделей, чтобы минимизировать ошибку.

### Что такое пошаговое аддитивное моделирование (Forward Stagewise Additive Modeling)?  
Рассмотрим базовую модель $b(x; r_m)$, управляемую параметром $r_m$.  
Параметр $\beta_m$ определяет вклад каждой слабой базовой модели в итоговый ансамбль.  

Итоговая модель $f(x)$, построенная на основе $M$ слабых моделей, описывается следующим образом:  
$$
f(x) = \sum_{m=1}^M \beta_m \cdot b(x; r_m)
$$  

Для получения классификатора используется функция знака $\text{sign}(x)$, преобразующая значения в классы:  
$$
G(x) = \text{sign}(f(x)) = \text{sign}\left(\sum_{m=1}^M \beta_m \cdot b(x; r_m)\right)
$$  

### Глобальная оптимизация для набора данных $D$  
Для набора данных $D$, содержащего $N$ наблюдений, минимизация функции потерь на всем наборе определяется следующим образом:  
$$
\min \sum_{i=1}^N \text{Loss}\left(y_i, \sum_{m=1}^M \beta_m \cdot b(x_i; r_m)\right)
$$  
На каждом шаге цель алгоритма — оптимально подобрать параметры $\beta_m$ и $r_m$ для текущей базовой модели так, чтобы приблизить решение к глобальному оптимуму:  
$$
\min \sum_{i=1}^N \text{Loss}\left(y_i, f_{m-1}(x_i) + \beta_m \cdot b(x_i; r_m)\right)
$$  

### Алгоритм пошагового аддитивного моделирования  
#### Входные данные:  
- Набор данных $D = \{(x_1, y_1), (x_2, y_2), \dots, (x_N, y_N)\}$  
- Функция потерь $\text{Loss}(y, f(x))$  
- Набор базовых моделей $\{b(x; r_m)\}$  

#### Выходные данные:  
- Итоговая модель $f(x)$.  

#### Шаги алгоритма:  
1. Инициализация: начальная модель $f_0(x) = 0$.  
2. Для каждого $m = 1, 2, \dots, M$:  
   - Минимизировать функцию потерь:  
$$
     (\beta_m, r_m) = \arg \min_{\beta, r} \sum_{i=1}^N \text{Loss}\left(y_i, f_{m-1}(x_i) + \beta \cdot b(x_i; r)\right)
$$  
   - Обновить модель:  
$$
     f_m(x) = f_{m-1}(x) + \beta_m \cdot b(x; r_m)
$$  
3. Итоговая модель задается следующим образом:  
$$
   f(x) = \sum_{m=1}^M \beta_m \cdot b(x; r_m)
$$  



### b. Что такое экспоненциальная функция потерь и почему она используется в алгоритме AdaBoost

Предположим, что $y$ принадлежит промежутку $(-1, 1)$, тогда экспоненциальная функция потерь имеет вид:

$$
\text{Loss}(y, f(x)) = \mathbb{E}(e^{-f(x)} \mid x) = P(y = 1 \mid x) e^{-f(x)} + P(y = -1 \mid x) e^{f(x)}
$$

Если мы возьмем производную от этой функции потерь по $f(x)$ и приравняем её к нулю, то получим, что при минимизации экспоненциальной функции потерь на самом деле происходит подгонка к логистической регрессии для вероятности $P(y = 1 \mid x)$:

$$
\frac{d}{df(x)} \mathbb{E}(e^{-f(x)}) = [-P(y = 1 \mid x) e^{-f(x)} + P(y = -1 \mid x) e^{f(x)}] = 0
$$

Решив это уравнение, мы получим:

$$
f(x) = \frac{1}{2} \log \frac{P(y = 1 \mid x)}{P(y = -1 \mid x)}
$$

Таким образом, оптимальное решение для $f(x)$ соответствует оптимальной вероятности по Байесу:

$$
\text{sign}(f(x)) = \begin{cases}
1, & \text{если} \, P(y = 1 \mid x) > P(y = -1 \mid x) \\
-1, & \text{если} \, P(y = 1 \mid x) < P(y = -1 \mid x)
\end{cases}
$$

Где:
- $\text{sign}(x) = 1$ при $x > 0$
- $\text{sign}(x) = -1$ при $x < 0$

Экспоненциальная функция потерь в AdaBoost помогает эффективно повышать вес правильно классифицированных объектов, способствуя тому, чтобы модель фокусировалась на сложных примерах, постепенно уменьшая ошибку на тренировочных данных.

### c. Математика за AdaBoost — как вычислить оптимальные параметры

Предположим, что после $m-1$ итераций мы вычислили функцию классификации $f_{m-1}(x)$, которая имеет вид:

$$
f_{m-1}(x) = f_{m-2}(x) + \beta_{m-1} b(x; r_{m-1}) = \sum_{i=1}^{m-1} \beta_i b(x_i)
$$

Теперь мы находимся на $m$-й итерации и хотим найти оптимальные параметры $\beta_m$ и $b_m(x; r_m)$ (обозначим $b_m(x)$) для минимизации экспоненциальной функции потерь.

Важно: выход $b_m(x)$ принадлежит промежутку $(-1, 1)$, а не вероятности. Целью является минимизация следующего выражения:

$$
\sum_{i=1}^N \arg \min_{\beta, b(x)} \text{Loss}(y_i, f_{m-1}(x) + \beta b(x))
$$

Где:

$$
\text{Loss}(y_i, f_{m-1}(x) + \beta b(x)) = \exp(-y_i f_{m-1}(x_i)) \cdot \exp(-y_i \beta b(x_i))
$$

Оптимизация осуществляется по $\beta$ и $b(x)$. Мы пытаемся минимизировать экспоненциальную потерю, вычисляя оптимальные веса для каждого примера:

$$
\min_{\beta, b(x)} \sum_{i=1}^N W_{mi} \exp(-y_i f_{m-1}(x_i)) \cdot \exp(-y_i \beta b(x_i))
$$

где $W_{mi} = \exp(-y_i f_{m-1}(x_i))$ — веса для каждого примера.

#### 1. Вычисление оптимального $b_m(x)$

Для нахождения оптимального $b_m(x)$ мы минимизируем следующее выражение:

$$
\arg \min_{b(x)} \sum_{i=1}^N W_{mi} \exp(-y_i \beta b(x_i))
$$

Решение для $b(x)$ даст нам значение, которое минимизирует экспоненциальную потерю для текущей итерации.

#### 2. Вычисление оптимального $\beta_m$

Затем для нахождения оптимального $\beta_m$, мы решаем:

$$
\arg \min_{\beta} \sum_{i=1}^N W_{mi} \exp(-y_i \beta b(x_i))
$$

Решение этого уравнения позволяет найти весовой коэффициент, который наилучшим образом сочетает информацию от предыдущих итераций с новыми классификаторами.

#### 3. Обновление весов $W_{m+1, i}$

После вычисления оптимальных значений $\beta_m$ и $b_m(x)$, обновляем веса для следующей итерации. Вес $W_{m+1, i}$ вычисляется как:

$$
W_{m+1, i} = \exp(-y_i f_m(x_i)) = \exp(-y_i [f_{m-1}(x_i) + \beta_m b_m(x_i)])
$$

Нормализуем веса, чтобы их сумма равнялась 1, что соответствует вероятности (аналогично функции softmax):

$$
W_{m+1, i} = \frac{W_{m+1, i}}{Z_m}
$$

где $Z_m$ — нормализующий фактор, обеспечивающий, что сумма всех весов равна 1.

Этот процесс повторяется для каждой итерации AdaBoost, с каждым разом улучшая классификацию и минимизируя ошибку.



### d. Актуальный рекуррентный алгоритм для AdaBoost с использованием деревьев

**Входные данные модели:**

Набор данных $D = \{(x_1, y_1), ..., (x_N, y_N)\}$, где $y_i \in \{-1, 1\}$.

**Выходные данные модели:**

Финальный классификатор $G(x)$.

**Шаги алгоритма:**

1. **Инициализация веса $T_1$:**

$$
   T_1 = (W_{11}, W_{12}, ..., W_{1N}), \quad W_{hi} \in \{1, 2, 3, 4, ..., N\}
$$

2. **Для $m = 1, 2, 3, ..., M$ (конечный классификатор состоит из $M$ слабых обучающих моделей):**

   - Используем набор данных $D$ с весами $T_m$ для обучения слабого классификатора $b_m(x)$, где $b_m(x) \in \{-1, 1\}$.

$$
     b_m = \arg \min \sum_{i=1}^{N} W_{mi} \cdot I(y_i \neq b_m(x_i))
$$
     Здесь $I$ — индикаторная функция, которая принимает значение 1, если $y_i \neq b_m(x_i)$, и 0 в противном случае.

   - Рассчитываем ошибку $e_m$ для $b_m(x)$ на наборе данных $D$:

$$
     e_m = \frac{\sum_{i=1}^{N} W_{mi} \cdot I(y_i \neq b_m(x_i))}{\sum_{i=1}^{N} W_{mi}}
$$

     Так как $\sum_{i=1}^{N} W_{mi} = 1$, получаем:

$$
     e_m = \sum_{i=1}^{N} W_{mi} \cdot I(y_i \neq b_m(x_i))
$$

   - Рассчитываем параметр $\beta_m$ для слабого классификатора $b_m(x)$:

$$
     \beta_m = \frac{1}{2} \log\left(\frac{1 - e_m}{e_m}\right)
$$

   - Обновляем веса $W_{m+1}$ для следующего слабого классификатора:

$$
     W_{m+1,i} = W_{mi} \cdot \exp(-\beta_m y_i b_m(x_i)), \quad Z_m = \sum_{i=1}^{N} W_{mi} \cdot \exp(-\beta_m y_i b_m(x_i))
$$

3. **Построение конечного классификатора:**

   Финальный классификатор $G(x)$ вычисляется как:

$$
   G(x) = \text{sign}\left(\sum_{m=1}^{M} \beta_m b_m(x)\right)
$$

   Где $\text{sign}$ — это функция знака, принимающая значение 1, если выражение положительно, и -1, если отрицательно.

### e. Подробное рассмотрение процесса обновления весов в AdaBoost

Помним, что:

$$
W_{m+1,i} = W_{mi} \cdot \exp(-\beta_m y_i b_m(x_i)), \quad Z_m = \sum_{i=1}^{N} W_{mi} \cdot \exp(-\beta_m y_i b_m(x_i))
$$

Таким образом, если $\beta_m > 0$, то происходит следующее:

- Если классификация выполнена правильно, то вес для соответствующего примера уменьшается, так как $\exp(-\beta_m)$ уменьшает значение веса.
- Если классификация выполнена неправильно, то вес для этого примера увеличивается, так как $\exp(\beta_m)$ увеличивает значение веса.

Это означает, что если классификатор правильно классифицирует пример, его вес уменьшается, что уменьшает его влияние на последующие классификации. Напротив, если классификатор ошибается, то вес примера увеличивается, что заставляет модель больше фокусироваться на этом примере в следующих итерациях.

### Применение в библиотеке Scikit-learn

**AdaBoostClassifier:**

```python
class sklearn.ensemble.AdaBoostClassifier(
    base_estimator=None,
    n_estimators=50,
    learning_rate=1.0,
    algorithm='SAMME.R',
    random_state=None
)
```

- **base_estimator**: объект (по умолчанию None) — это базовый классификатор, на основе которого строится ансамбль. По умолчанию используется классификатор DecisionTreeClassifier с глубиной максимума 1. Вы можете также использовать другие модели машинного обучения, такие как SVC.

- **n_estimators**: целое число (по умолчанию 50) — максимальное количество классификаторов, при котором обучение будет завершено. Это соответствует $M$ в формуле.

- **learning_rate**: вещественное число (по умолчанию 1.0) — коэффициент обучения, который уменьшает вклад каждого классификатора. Например, если предсказание на предыдущем шаге было $f_{m-1} = 1$, коэффициент обучения $\text{learning_rate} = 0.1$, а коррекция для следующего дерева $= 0.5$, то обновленное предсказание будет равно:

  $$
  f_m = 1 + 0.1 \cdot 0.5 = 1.05
  $$

  Уменьшение коэффициента обучения замедляет процесс обучения, но может привести к лучшему качеству модели.

- **algorithm**: строка, значения ‘SAMME’ или ‘SAMME.R’ (по умолчанию ‘SAMME.R’) — алгоритм, который будет использован для бустинга. Если выбран ‘SAMME.R’, то применяется реальный алгоритм бустинга. Базовый классификатор должен поддерживать вычисление вероятностей классов. Если выбран ‘SAMME’, используется дискретный алгоритм бустинга. Алгоритм ‘SAMME.R’ обычно сходится быстрее и достигает меньшей ошибки на тесте при меньшем количестве итераций бустинга.


## GBM (Gradient Boosting Machine)

**Однострочное описание**: Постепенное добавление слабых базовых моделей для аппроксимации отрицательного градиента с целью уменьшения общей ошибки.

### a. Различия между AdaBoost и GBM
AdaBoost использует экспоненциальную функцию потерь, при этом экспоненциальная потеря растет экспоненциально для отрицательных значений, что делает метод более чувствительным к выбросам. В отличие от этого, GBM позволяет использовать более устойчивые функции потерь, при условии, что они являются непрерывно дифференцируемыми.

| Модели | Методы корректировки предыдущих ошибок |
|--------|----------------------------------------|
| **AdaBoost** | Добавление весов для неправильно классифицированных примеров и уменьшение весов для правильно классифицированных примеров. |
| **GBM** | Использование отрицательного градиента как индикатора ошибок, сделанных предыдущими базовыми моделями, и обучение следующей базовой модели для аппроксимации отрицательного градиента предыдущих моделей. |

### b. Отрицательный градиент в GBM

В AdaBoost мы упоминаем метод Forward Stagewise Additive Modeling. Предположим, что мы находимся на m-ой итерации:

$$
f_m(x) = f_{m-1} + B_m b_m(x)
$$

Мы хотим уменьшить ошибку, как в AdaBoost:

$$
\min \sum_{i=1}^{N} \text{Loss}(y_i, f_m(x)) \quad \text{или} \quad \min \sum_{i=1}^{N} \text{Loss}(y_i, f_{m-1}(x) + B_m \cdot b_m(x))
$$

Однако здесь задача отличается от случая в AdaBoost. В AdaBoost мы точно знаем функцию потерь (экспоненциальную функцию), поэтому можем найти оптимальное значение $b_m(x)$. В GBM же мы хотим работать с любой функцией потерь, которая является дифференцируемой. Для этого мы применяем идею, аналогичную методу градиентного спуска, чтобы найти оптимальное значение $b_m(x)$ с помощью отрицательного градиента:

$$
\min \sum_{i=1}^{N} \text{Loss}(y_i, f_{m-1}(x) + \alpha_m \cdot b_m(x)) \quad \Rightarrow \quad b_m(x) = \text{const} \cdot \nabla \text{Loss}(y_i, f_{m-1}(x))
$$

Здесь $\alpha_m$ — параметр, похожий на скорость обучения, но принимающий отрицательные значения.

### c. Алгоритм GBM

**Входные данные модели**: Набор данных $D = \{(x_1, y_1), \dots, (x_N, y_N)\}, y_i \in \{-1, 1\}$

**Выходные данные модели**: Финальный классификатор/регрессор $f_m(x)$

**Шаги**:

1. **Инициализация**:
$$
   f_0(x) = \arg \min \sum_{i=1}^{N} \text{Loss}(y_i, 7)
$$

2. Для $m = 1, 2, 3, \dots, M$:
   - Вычисление отрицательного градиента:
$$
     \nabla \text{Loss}(y_i, f_{m-1}(x)) \quad \text{для} \quad i = 1, 2, 3, \dots, N
$$
   - Обучение новой модели (например, дерева) для минимизации квадратных потерь:
$$
     b_m(x) = \arg \min \sum_{i=1}^{N} \left( y_i - b(x) \right)^2
$$
   - Использование линейного поиска для нахождения оптимального шага (аналогично концепции скорости обучения в SGD):
$$
     \alpha_m = \arg \min \sum_{i=1}^{N} \text{Loss}(y_i, f_{m-1}(x_i) + \alpha_m b_m(x_i))
$$
   - Обновление функции $f_m(x)$:
$$
     f_m(x) = f_{m-1}(x) + \alpha_m \cdot b_m(x)
$$

3. Для $m = 1, 2, 3, \dots, M$:
$$
   f_m(x) = f_0(x) + \sum_{m=1}^M \alpha_m \cdot b_m(x)
$$


### d. Алгоритм регрессии на основе дерева решений GBM (Gradient Boosting Machine)
**Вход модели:** Данные D: $D = \{(x_1, y_1), \dots, (x_N, y_N)\}, y_i \in \mathbb{R}$  
**Выход модели:** Финальный регрессор: $f_M(x)$  
**Функция потерь:** Квадратичная ошибка (Square Loss)  
**Шаги алгоритма:**

1. **Инициализация:**
   Начальное значение $f_0(x)$ выбирается как решение задачи минимизации функции потерь:
$$
   f_0(x) = \arg\min \sum_{i=1}^N (y_i - f_0(x_i))^2
$$

2. **Для $m = 1, 2, 3, \dots, M$:**
   - **Вычисление отрицательного градиента** для функции потерь:
$$
     g_m(x) = -\frac{\partial}{\partial f_{m-1}(x)} \text{Loss}(y_i, f_{m-1}(x_i)) = 2(y_i - f_{m-1}(x_i))
$$
   - **Обучение нового дерева решений (CART)** для минимизации квадратичной ошибки:
     Построение дерева с разделением области значений на $J$ частей $R_{j,m}$ для каждого шага $m$.
$$
     b_m(x) = \arg\min_{b(x)} \sum_{i=1}^N (y_i - b(x_i))^2
$$
   - Вместо поиска оптимального параметра $\theta$ для всего дерева, оптимальные параметры $\theta_{j,m}$ находятся для каждой зоны дерева отдельно:
$$
     \theta_{j,m} = \arg\min \sum_{i=1}^N \left( y_i - (f_{m-1}(x_i) + \theta_{j,m} I(x_i \in R_{j,m})) \right)^2
$$
   - **Обновление функции предсказания**:
$$
     f_m(x) = f_{m-1}(x) + \sum_{j=1}^J \theta_{j,m} I(x \in R_{j,m})
$$

3. **Вывод финальной модели:**
$$
   f(x) = f_0(x) + \sum_{m=1}^M \sum_{j=1}^J \theta_{j,m} I(x \in R_{j,m})
$$

### e. Алгоритм классификации на основе дерева решений GBM
**Вход модели:** Данные D: $D = \{(x_1, y_1), \dots, (x_N, y_N)\}, y_i \in \{-1, 1\}$  
**Выход модели:** Финальный классификатор: $f_M(x)$  
**Функция потерь:** Девиантность (Deviance Loss)  
**Шаги алгоритма:**

1. **Инициализация:**
   - **Функция потерь (девиантность)** для каждого элемента:
$$
     p(y_i = 1 | x_i) = \frac{1}{1 + \exp(-f_m(x_i))}
$$
   - Начальные значения $f_0(x)$ выбираются так, чтобы минимизировать функцию потерь:
$$
     f_0(x) = \arg\min \sum_{i=1}^N \text{Loss}(p(y_i | x_i), y_i)
$$
   - Для нахождения оптимального $f_0(x)$ решается уравнение:
$$
     \sum_{i=1}^N \left( -\left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right] \right) = 0
$$

2. **Для $m = 1, 2, 3, \dots, M$:**
   - Вычисление отрицательного градиента для девиантности:
$$
     g_m(x_i) = \frac{d}{df_{m-1}(x_i)} \left[ y_i \log(p_{m-1}(x_i)) + (1 - y_i) \log(1 - p_{m-1}(x_i)) \right]
$$
   - **Обучение нового дерева решений (CART)** для минимизации девиантности:
$$
     b_m(x) = \arg\min \sum_{i=1}^N (y_i - b(x_i))^2
$$
   - Вместо поиска параметра $\theta$ для всего дерева, параметры $\theta_{j,m}$ оптимизируются для каждой зоны дерева:
$$
     \theta_{j,m} = \arg\min \sum_{i=1}^N \left( y_i - (f_{m-1}(x_i) + \theta_{j,m} I(x_i \in R_{j,m})) \right)^2
$$
   - **Обновление функции предсказания**:
$$
     f_m(x) = f_{m-1}(x) + \sum_{j=1}^J \theta_{j,m} I(x \in R_{j,m})
$$

3. **Вывод финальной модели:**
$$
   f(x) = f_0(x) + \sum_{m=1}^M \sum_{j=1}^J \theta_{j,m} I(x \in R_{j,m})
$$

### Применение в Scikit-learn

**GradientBoostingRegressor:**
```python
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(
    loss='ls',                # Тип функции потерь (по умолчанию 'ls' - минимизация квадратичной ошибки)
    learning_rate=0.1,        # Коэффициент обучения
    n_estimators=100,         # Количество деревьев
    subsample=1.0,            # Фракция случайно выбранных обучающих данных для каждого базового дерева
    max_depth=3,              # Глубина деревьев
    random_state=None,        # Начальное состояние для воспроизводимости результатов
    alpha=0.9,                # Альфа-квантиль для Huber loss, если он выбран
    verbose=0,                # Отключение вывода
    warm_start=False,         # Разрешить добавление деревьев в обученную модель
)
```

Основные гиперпараметры для **GradientBoostingRegressor** описаны выше. Стоит отметить, что выбор гиперпараметров, таких как коэффициент обучения (`learning_rate`), количество деревьев (`n_estimators`), и выбор функции потерь (`loss`), сильно влияет на производительность модели.


